# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,allapalli,19.4167,80.0667,21.56,85,88,0.27,IN,1701291299
1,1,portland,45.5234,-122.6762,6.99,67,75,1.54,US,1701291219
2,2,ovria,38.1833,21.7167,17.29,81,93,2.72,GR,1701291300
3,3,laguna,38.4210,-121.4238,17.88,47,100,3.09,US,1701291300
4,4,hamilton,39.1834,-84.5333,8.83,52,0,7.20,US,1701291300


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng" 
    , "Lat" 
    , geo = True 
    , size = "Humidity"
    , color = "City"
    , tiles = 'OSM'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df.loc[(city_data_df['Max Temp'] < 24) & (city_data_df['Max Temp'] > 20) & (city_data_df['Wind Speed'] <4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
ideal_weather_cities = ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
169,169,sur,22.5667,59.5289,23.90,67,0,2.57,OM,1701291353
178,178,taoudenni,22.6783,-3.9836,21.68,12,0,3.60,ML,1701291356
444,444,gebeit,21.0667,36.3167,20.99,81,0,2.62,SD,1701291437
559,559,guskhara,23.5000,87.7500,20.70,61,0,2.32,IN,1701291471


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

C:\Users\Franc\AppData\Local\Temp\ipykernel_12760\3614469063.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
169,sur,OM,22.5667,59.5289,67,
178,taoudenni,ML,22.6783,-3.9836,12,
444,gebeit,SD,21.0667,36.3167,81,
559,guskhara,IN,23.5000,87.7500,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
 'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':20
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sur - nearest hotel: Sur Hotel
taoudenni - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
guskhara - nearest hotel: Guskara Municipal Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
169,sur,OM,22.5667,59.5289,67,Sur Hotel
178,taoudenni,ML,22.6783,-3.9836,12,No hotel found
444,gebeit,SD,21.0667,36.3167,81,No hotel found
559,guskhara,IN,23.5000,87.7500,61,Guskara Municipal Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)